In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate


from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


chat = ChatOpenAI(openai_api_key=api_key,
                  temperature=0.1,
                  streaming=True,
                  callbacks=[
                      StreamingStdOutCallbackHandler(),
                      ],)

examples = [
    {
        "question": "영화, 어벤져스: 엔드게임에 대해 알려주세요.",
        "answer": """
        제목: 어벤져스: 엔드게임
        감독: 안소니 루소, 조 루소
        주요 출연진: 로버트 다우니 주니어, 크리스 에반스, 스칼렛 요한슨
        예산: $356,000,000
        흥행 수익: $2,798,000,000
        장르: 액션, SF, 슈퍼히어로
        시놉시스: 타노스가 우주의 절반을 소멸시킨 후, 어벤져스는 그를 물리치고 사라진 사람들을 되돌리기 위한 마지막 전투에 나선다.
        """,
    },
    {
        "question": "영화, 라라랜드에 대해 알려주세요.",
        "answer": """
        제목: 라라랜드
        감독: 데이미언 셔젤
        주요 출연진: 라이언 고슬링, 엠마 스톤
        예산: $30,000,000
        흥행 수익: $448,914,993
        장르: 뮤지컬, 드라마, 로맨스
        시놉시스: 재즈 피아니스트 세바스찬과 배우 지망생 미아는 로스앤젤레스에서 꿈을 이루기 위해 노력하며 사랑에 빠지지만, 그들의 꿈과 사랑은 복잡하게 얽히게 된다.
        """,
    },
    {
        "question": "영화, 기생충에 대해 알려주세요.",
        "answer": """
        제목: 기생충
        감독: 봉준호
        주요 출연진: 송강호, 이선균, 조여정
        예산: $11,000,000
        흥행 수익: $258,800,000
        장르: 드라마, 스릴러
        시놉시스: 가난한 기택 가족은 부유한 박 사장의 집에서 일을 시작하게 되면서 점차 그들의 삶에 깊숙이 침투하게 된다. 하지만 그들 사이의 비밀이 드러나며 예기치 못한 사건이 벌어진다.
        """,
    },
]


example_prompt = PromptTemplate.from_template("human: {question}\nai: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    prefix="system : 당신은 영화를 일년에 최소 천개의 영화를 보고 평가하는 평론가 입니다.",
    suffix="human : 영화, {movie_name}에 대해 알려주세요.",
    input_variables=["movie_name"]
)

chain = prompt | chat

chain.invoke({'movie_name': '모아나'})

ai: 
        제목: 모아나
        감독: 론 클레멘츠, 존 머스커
        주요 출연진: 알리 크라발로, 드웨인 존슨
        예산: $150,000,000
        흥행 수익: $690,800,000
        장르: 애니메이션, 모험, 코미디
        시놉시스: 타피티 섬의 족장딸 모아나는 전설 속의 섬을 찾아 나서는 모험을 떠나며 자신의 운명을 찾아가는 이야기를 그린 애니메이션 영화이다.

AIMessageChunk(content='ai: \n        제목: 모아나\n        감독: 론 클레멘츠, 존 머스커\n        주요 출연진: 알리 크라발로, 드웨인 존슨\n        예산: $150,000,000\n        흥행 수익: $690,800,000\n        장르: 애니메이션, 모험, 코미디\n        시놉시스: 타피티 섬의 족장딸 모아나는 전설 속의 섬을 찾아 나서는 모험을 떠나며 자신의 운명을 찾아가는 이야기를 그린 애니메이션 영화이다.')

In [3]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human",  "{question}",),  
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화를 일년에 최소 천개의 영화를 보고 평가하는 평론가 입니다. "),
    example_prompt,
    ("human", "{movie_name}에 대해 알려주세요.",)
    
])

chain = final_prompt | chat

chain.invoke({"movie_name": "모아나"})


        제목: 모아나
        감독: 론 클레멘츠, 존 머스커
        주요 출연진: 알리 크라발로, 드웨인 존슨
        예산: $150,000,000
        흥행 수익: $690,800,000
        장르: 애니메이션, 모험, 코미디
        시놉시스: 타피티 섬의 어린 공주 모아나는 전설 속의 섬으로 가서 세계를 구하기 위한 모험을 떠난다. 함께 여정을 떠난 데미곤, 바다의 신과의 만남, 그리고 자신의 용기를 발견하게 된다.
        

AIMessageChunk(content='\n        제목: 모아나\n        감독: 론 클레멘츠, 존 머스커\n        주요 출연진: 알리 크라발로, 드웨인 존슨\n        예산: $150,000,000\n        흥행 수익: $690,800,000\n        장르: 애니메이션, 모험, 코미디\n        시놉시스: 타피티 섬의 어린 공주 모아나는 전설 속의 섬으로 가서 세계를 구하기 위한 모험을 떠난다. 함께 여정을 떠난 데미곤, 바다의 신과의 만남, 그리고 자신의 용기를 발견하게 된다.\n        ')

In [4]:
chain.invoke({"movie_name": "파수꾼"})

죄송합니다. 제가 본 영화 중에 '파수꾼'에 대한 정보는 없습니다. 다른 영화에 대해 궁금한 점이 있으시면 언제든지 물어보세요. 저는 최대한 도와드리겠습니다.

AIMessageChunk(content="죄송합니다. 제가 본 영화 중에 '파수꾼'에 대한 정보는 없습니다. 다른 영화에 대해 궁금한 점이 있으시면 언제든지 물어보세요. 저는 최대한 도와드리겠습니다.")

In [5]:
chain.invoke({"movie_name": "고지전"})

죄송합니다, 제가 본 영화 중에 '고지전'이라는 영화는 없습니다. 다른 영화에 대해 알고 계신다면 다시 한 번 질문해주시면 해당 영화에 대해 알려드릴 수 있을 것입니다. 혹은 다른 질문이 있으시면 언제든지 물어보세요!

AIMessageChunk(content="죄송합니다, 제가 본 영화 중에 '고지전'이라는 영화는 없습니다. 다른 영화에 대해 알고 계신다면 다시 한 번 질문해주시면 해당 영화에 대해 알려드릴 수 있을 것입니다. 혹은 다른 질문이 있으시면 언제든지 물어보세요!")

In [6]:
chain.invoke({"movie_name": "Black Night"})

죄송합니다, "Black Night"라는 영화에 대한 정보를 찾을 수 없습니다. 혹시 제목이 오타가 있을 수도 있습니다. 다시 한번 확인해 주시고, 영화 제목이 정확한지 다시 알려주시면 해당 영화에 대한 정보를 제공해 드리겠습니다.

AIMessageChunk(content='죄송합니다, "Black Night"라는 영화에 대한 정보를 찾을 수 없습니다. 혹시 제목이 오타가 있을 수도 있습니다. 다시 한번 확인해 주시고, 영화 제목이 정확한지 다시 알려주시면 해당 영화에 대한 정보를 제공해 드리겠습니다.')